In [1]:
# Install dependencies
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Retrieve PDF to google colab resipotory
# The following paper talks about VideoTaskformer
#!curl -o paper.pdf https://arxiv.org/pdf/2303.13519.pdf
# My original paper on a stochastic heat equation is also uploaded to the Colab resipotory

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6745k  100 6745k    0     0   106k      0  0:01:03  0:01:03 --:--:--  154k


In [3]:
# Import dependencies
from langchain.document_loaders import PyPDFLoader # read PDF
from langchain.embeddings import OpenAIEmbeddings # use to index the information in documents
from langchain.vectorstores import Chroma # vectorization of texts
from langchain.chains import ChatVectorDBChain # communicate with vectorized texts
from langchain.llms import OpenAI # connect to ChatGPT


In [30]:
# Read the paper using PyPDFLoader
pdf_path = "./20-EJP415.pdf"#"./paper.pdf" # set path
loader = PyPDFLoader(pdf_path) # load pdf
pages = loader.load_and_split() # load pdf content
print(pages[0].page_content) # print the first page

ElectronicJournal
o
f
P
robability
Electron. J. Probab. 25(2020), no. 5, 1–38.
ISSN: 1083-6489 https://doi.org/10.1214/20-EJP415
On the peaks of a stochastic heat equation on a sphere
with a large radius
Weicong Su*
Abstract
For everyR> 0, consider the stochastic heat equation
∂tuR(t,x) =1
2∆S2
RuR(t,x) +σ(uR(t,x))ξR(t,x)
onS2
R, whereξR=˙WRare centered Gaussian noises with the covariance struc-
ture given by E[˙WR(t,x)˙WR(s,y)] =hR(x,y)δ0(t−s), wherehRis symmetric and
semi-positive deﬁnite and there exist some ﬁxed constants −2< C hup<2and
1
2Chup−1<Chdown⩽Chupsuch that for all R> 0andx,y∈S2
R,(logR)Chdown/2=
hdown(R)⩽hR(x,y)⩽hup(R) = (logR)Chup/2,∆S2
Rdenotes the Laplace-Beltrami
operator deﬁned on S2
Randσ:R↦→Ris Lipschitz continuous, positive and uni-
formly bounded away from 0and∞. Under the assumption that uR,0(x) =uR(0,x)
is a nonrandom continuous function on x∈S2
Rand the initial condition that there
exists a ﬁnite positive Usuch that supR>0supx∈S2
R|uR,0(x)|⩽U, we prove that f

In [31]:
# Set OPENAI API keys
import os
os.environ["OPENAI_API_KEY"] = #'YOU_SHALL_NOT_SEE_THIS'



In [32]:
# Transform PDF to a vector dababase
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectordb.persist()

###Ask questions about the PDF and get answer from chat models.

Select a chat model.

In [33]:
#https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions
# select the correct model; gpt-3.5-turbo would not work! model not supported error shows up as in the link
pdf_qa = ChatVectorDBChain.from_llm(OpenAI(temperature=0.9, model="gpt-3.5-turbo-instruct"), vectordb,\
                                    return_source_documents=True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/conversational_retrieval/base.py:409: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warnings.warn(


The summarization of the paper is correct.


In [34]:
query = "What is the paper about?"
result = pdf_qa({"question":query, "chat_history":""})
print("Answer:")
print(result["answer"])

Answer:

The paper is about the peaks of a stochastic heat equation on a sphere, focusing on the asymptotic behavior of the solution. It uses a stochastic partial differential equation and a discretization technique to give an asymptotic lower bound of the supremum of the solution. 


When asked about the important theorems or results of the paper, the chat model cannot give a correct answer.

In [35]:
query = "What are the main results of this paper?"
result = pdf_qa({"question":query, "chat_history":""})
print("Answer:")
print(result["answer"])

Answer:


The main results of this paper include comparing different approaches for downstream tasks, finding improvements in performance with the use of task labels and visual features, and providing additional context by training on entire videos rather than individual clips. These techniques were found to be effective in achieving higher accuracy on various tasks such as step classification, procedural activity recognition, and short-term and long-term forecasting.


Let the chat model retrieve a basic information about the paper: the name of the author. The model seems to be self-contradictory.

In [36]:
query = "What is the author of this paper?"
result = pdf_qa({"question":query, "chat_history":""})
print("Answer:")
print(result["answer"])

Answer:
 The author of this paper is not mentioned in the context provided. We cannot determine the author based on this information. 



In [38]:
query = "What are the names appeared in this paper?"
result = pdf_qa({"question":query, "chat_history":""})
print("Answer:")
print(result["answer"])

Answer:
 Christian Szegedy, Vincent Vanhoucke, Sergey Ioffe, Jon Shlens, Zbigniew Wojna, Yansong Tang, Dajun Ding, Yongming Rao, Yu Zheng, Danyang Zhang, Lili Zhao, Jiwen Lu, Jie Zhou, Qingyun Wang, Manling Li, Hou Pong Chan, Lifu Huang, Julia Hockenmaier, Girish Chowdhary, Heng Ji, Hu Xu, Gargi Ghosh, Po-Yao Huang, Dmytro Okhonko, Armen Aghajanyan, Florian Metze, Luke Zettlemoyer, Christoph Feichtenhofer, Zhongwen Xu, Yi Yang, Alex G Hauptmann, Luowei Zhou, Chenliang Xu, Jason J Corso, Dimitri Zhukov, Jean-Baptiste Alayrac, Ramazan Gokberk Cinbis, David Fouhey, Ivan Laptev, Josef Sivic, Steven Rosenberg, T. Funaki, S. Peszat, S. Tindel, Z. Brze´ zniak, B. Goldys, Q. T. Le Gia


In [39]:
query = "Does the paper mention Weicong Su?"
result = pdf_qa({"question":query, "chat_history":""})
print("Answer:")
print(result["answer"])

Answer:
 Yes, the paper mentions Weicong Su as the author of the paper.
